# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(98120900, 138)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_num_happend_later_mean,qou_num_happend_before_mean,qou_happend_later_ratio_mean,qou_last_interact_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,qou_num_happend_later_sqrt_num_cousers,qou_num_happend_before_sqrt_num_cousers,qou_happend_later_ratio_sqrt_num_cousers,qou_last_interact_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,cointeractions_coef_clicks_clicks,cointeractions_coef_clicks_carts,cointeractions_coef_clicks_orders,cointeractions_coef_carts_clicks,cointeractions_coef_carts_carts,cointeractions_coef_carts_orders,cointeractions_coef_orders_clicks,cointeractions_coef_orders_carts,cointeractions_coef_orders_orders,cointeractions_time_decay_clicks_clicks,cointeractions_time_decay_clicks_carts,cointeractions_time_decay_clicks_orders,cointeractions_time_decay_carts_clicks,cointeractions_time_decay_carts_carts,cointeractions_time_decay_carts_orders,cointeractions_time_decay_orders_clicks,cointeractions_time_decay_orders_carts,cointeractions_time_decay_orders_orders,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean,item_item_cnt,item_item_clicks_cnt,item_item_carts_cnt,item_item_orders_cnt,item_item_total_actions_cnt,item_item_clicks_ratio,item_item_carts_ratio,item_item_orders_ratio
0,11851599.0,12946375.0,6070.0,338.0,85.0,3648.0,2.415886e-05,2939.0,159.0,43.0,1757.0,2.682789e-05,418.0,21.0,4.0,320.0,31.0,10.0,290.0,11.0,3.0,312.0,15.0,3.0,254.0,14.0,4.0,278.0,17.0,4.0,411.0,16.0,3.0,20.995497,0.054352,2781.836426,2604.236328,143.123062,34.477066,20.941145,0.000000,0.053747,0.000000,0.0,5.064732e-03,0.000000,0.0,0.049832,0.576574,0.230504,0.000000,1.655695,0.924159,0.052304,4.912245e-03,0.050656,4.739619e-03,0.053462,5.034503e-03,0.052710,4.955230e-03,0.053699,5.059691e-03,0.053648,5.054274e-03,0.048120,4.476359e-03,0.053728,5.062792e-03,0.052376,4.919546e-03

# Train params

In [7]:
from meow_utils import *
from catboost.utils import eval_metric

PARAMS = {
    'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20;border=0.5',
    'use_best_model': True,
    'thread_count': -1,
}
N_SPLITS = 5
NUM_SAMPLES = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '10'

# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [10]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  cnt = 0
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    if (users[i], items[i]) in tar_small_index:
      label.append(1)
      cnt += 1
    else: 
      label.append(0)
  labels.append(label)
  print(f'Found {cnt} positives', end = ' ')
del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 753055 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 192092 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 144328 positives 

In [11]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, N_SPLITS, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 20]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])

        params = {'thread_count' : os.cpu_count(),
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [13]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type, draw_importance = False)

remove old version
Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0
0:	learn: 0.9998224	test: 0.8885338	best: 0.8885338 (0)	total: 2.94s	remaining: 48m 53s
1:	learn: 0.9999315	test: 0.9089125	best: 0.9089125 (1)	total: 5.27s	remaining: 43m 48s
2:	learn: 0.9999613	test: 0.9153048	best: 0.9153048 (2)	total: 7.82s	remaining: 43m 19s
3:	learn: 0.9999673	test: 0.9190285	best: 0.9190285 (3)	total: 10.1s	remaining: 41m 56s
4:	learn: 0.9999702	test: 0.9192664	best: 0.9192664 (4)	total: 12.3s	remaining: 40m 53s
5:	learn: 0.9999722	test: 0.9213760	best: 0.9213760 (5)	total: 15s	remaining: 41m 31s
6:	learn: 0.9999722	test: 0.9220581	best: 0.9220581 (6)	total: 17.8s	remaining: 42m 8s
7:	learn: 0.9999782	test: 0.9228908	best: 0.9228908 (7)	total: 20.6s	remaining: 42m 34s
8:	learn: 0.9999762	test: 0.9234619	best: 0.9234619 (8)	total: 22.8s	remaining: 41m 49s
9:	learn: 0.9999782	test: 0.9238386	best: 0.9238386 (9)	total: 24.9s	remaining: 41m 8s
10:	learn: 0.9999792	test: 0.9242550	best: 0.9242550

Training has stopped (degenerate solution on iteration 41, probably too small l2-regularization, try to increase it)



bestTest = 0.9970831945
bestIteration = 32

Shrink model to first 33 iterations.
FOLD: 1 Sample: 0
0:	learn: 0.9999973	test: 0.9949895	best: 0.9949895 (0)	total: 888ms	remaining: 14m 47s
1:	learn: 0.9999974	test: 0.9963008	best: 0.9963008 (1)	total: 1.73s	remaining: 14m 24s
2:	learn: 0.9999973	test: 0.9966755	best: 0.9966755 (2)	total: 2.58s	remaining: 14m 18s
3:	learn: 0.9999974	test: 0.9967953	best: 0.9967953 (3)	total: 3.45s	remaining: 14m 18s
4:	learn: 0.9999974	test: 0.9969764	best: 0.9969764 (4)	total: 4.31s	remaining: 14m 16s
5:	learn: 0.9999974	test: 0.9969875	best: 0.9969875 (5)	total: 5.33s	remaining: 14m 43s
6:	learn: 0.9999974	test: 0.9970959	best: 0.9970959 (6)	total: 6.42s	remaining: 15m 10s
7:	learn: 0.9999974	test: 0.9971232	best: 0.9971232 (7)	total: 7.46s	remaining: 15m 25s
8:	learn: 0.9999974	test: 0.9971571	best: 0.9971571 (8)	total: 8.53s	remaining: 15m 39s
9:	learn: 0.9999974	test: 0.9971871	best: 0.9971871 (9)	total: 9.58s	remaining: 15m 48s
10:	learn: 0.9999974

Training has stopped (degenerate solution on iteration 34, probably too small l2-regularization, try to increase it)



bestTest = 0.9973315179
bestIteration = 21

Shrink model to first 22 iterations.
FOLD: 1 Sample: 1
0:	learn: 0.9999973	test: 0.9949432	best: 0.9949432 (0)	total: 1.11s	remaining: 18m 34s
1:	learn: 0.9999972	test: 0.9962331	best: 0.9962331 (1)	total: 1.92s	remaining: 16m
2:	learn: 0.9999972	test: 0.9967398	best: 0.9967398 (2)	total: 2.76s	remaining: 15m 16s
3:	learn: 0.9999972	test: 0.9968750	best: 0.9968750 (3)	total: 3.63s	remaining: 15m 2s
4:	learn: 0.9999973	test: 0.9969995	best: 0.9969995 (4)	total: 4.66s	remaining: 15m 26s
5:	learn: 0.9999973	test: 0.9970533	best: 0.9970533 (5)	total: 5.68s	remaining: 15m 40s
6:	learn: 0.9999973	test: 0.9971280	best: 0.9971280 (6)	total: 6.75s	remaining: 15m 57s
7:	learn: 0.9999973	test: 0.9972215	best: 0.9972215 (7)	total: 7.7s	remaining: 15m 55s
8:	learn: 0.9999972	test: 0.9972032	best: 0.9972215 (7)	total: 8.64s	remaining: 15m 51s
9:	learn: 0.9999971	test: 0.9972371	best: 0.9972371 (9)	total: 9.57s	remaining: 15m 47s
10:	learn: 0.9999971	test: